# Pandas basics 

Hi! In this programming assignment you need to refresh your `pandas` knowledge. You will need to do several [`groupby`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html)s and [`join`]()`s to solve the task. 

In [11]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline

In [12]:
!ls

Week 1 pandas Practice.ipynb           competitive-data-science-final-project
Week1 Recap of ML algorithms.ipynb


In [13]:
!ls competitive-data-science-final-project/

item_categories.csv      sales_train.csv.gz       test.csv.gz
item_cats.csv            sample_submission.csv.gz
items.csv                shops.csv


In [14]:
DATA_FOLDER = './competitive-data-science-final-project'

os.path.join(DATA_FOLDER, 'items.csv')

'./competitive-data-science-final-project/items.csv'

In [15]:
transactions = pd.read_csv(os.path.join(DATA_FOLDER, 'sales_train.csv.gz'))
items = pd.read_csv(os.path.join(DATA_FOLDER, 'items.csv'))
item_categories =  pd.read_csv(os.path.join(DATA_FOLDER, 'item_categories.csv'))
shops = pd.read_csv(os.path.join(DATA_FOLDER, 'shops.csv'))

In [16]:
#note that pd.read_csv can open a .gz file

Let's start with a simple task. 

<ol start="0">
  <li><b>Print the shape of the loaded dataframes and use [`df.head`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.head.html) function to print several rows. Examine the features you are given.</b></li>
</ol>

In [17]:
frames = [transactions, items, item_categories, shops]
titles = ['transactions', 'items', 'item_categories', 'shops']

zipped = zip(titles, frames)

for title, df in zipped:
    print('Shape of ', title, 'is ',df.shape)
    print(df.head(),'\n')

Shape of  transactions is  (2935849, 6)
         date  date_block_num  shop_id  item_id  item_price  item_cnt_day
0  02.01.2013               0       59    22154      999.00           1.0
1  03.01.2013               0       25     2552      899.00           1.0
2  05.01.2013               0       25     2552      899.00          -1.0
3  06.01.2013               0       25     2554     1709.05           1.0
4  15.01.2013               0       25     2555     1099.00           1.0 

Shape of  items is  (22170, 3)
                                           item_name  item_id  \
0          ! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.)         D        0   
1  !ABBYY FineReader 12 Professional Edition Full...        1   
2      ***В ЛУЧАХ СЛАВЫ   (UNV)                    D        2   
3    ***ГОЛУБАЯ ВОЛНА  (Univ)                      D        3   
4        ***КОРОБКА (СТЕКЛО)                       D        4   

   item_category_id  
0                40  
1                76  
2                40  
3   

Now use your `pandas` skills to get answers for the following questions. 
The first question is:

1. ** What was the maximum total revenue among all the shops in September, 2014?** 


* Hereinafter *revenue* refers to total sales minus value of goods returned.

*Hints:*

* Sometimes items are returned, find such examples in the dataset. 
* It is handy to split `date` field into [`day`, `month`, `year`] components and use `df.year == 14` and `df.month == 9` in order to select target subset of dates.
* You may work with `date` feature as with strings, or you may first convert it to `pd.datetime` type with `pd.to_datetime` function, but do not forget to set correct `format` argument. 

In [45]:
#can do this two ways: i) use the date block number or ii) follow their suggestion and split the date field

transactions['day'] = pd.to_numeric(transactions['date'].str.split('.').str.get(0))
transactions['month'] = pd.to_numeric(transactions['date'].str.split('.').str.get(1))
transactions['year'] = pd.to_numeric(transactions['date'].str.split('.').str.get(2))

df = transactions.drop(columns='date')

In [46]:
df.head()

,date_block_num,shop_id,item_id,item_price,item_cnt_day,day,month,year
0,0,59,22154,999.00,1.0,2,1,2013
1,0,25,2552,899.00,1.0,3,1,2013
2,0,25,2552,899.00,-1.0,5,1,2013
3,0,25,2554,1709.05,1.0,6,1,2013
4,0,25,2555,1099.00,1.0,15,1,2013


In [48]:
filter_ = (df.month == 9) & (df.year == 2014)

df['revenue'] = (df['item_price']*df['item_cnt_day'])

#max_store_revenue = df.groupby('shop_id')['item_sales'].sum().sort_values(ascending=False).iloc[0]#
max_store_revenue = df[filter_].groupby('shop_id')['revenue'].sum().sort_values(ascending=False).iloc[0]
print("The value of the total monthly revenue for the store with the largest value of this quantity is \n$", max_store_revenue)

The value of the total monthly revenue for the store with the largest value of this quantity is 
$ 7982852.199999956


Great! Let's move on and answer another question:

<ol start="2">
  <li><b>What item category generated the highest revenue in summer 2014?</b></li>
</ol>

* Submit `id` of the category found.
    
* Here we call "summer" the period from June to August.

*Hints:*

* Note, that for an object `x` of type `pd.Series`: `x.argmax()` returns **index** of the maximum element. `pd.Series` can have non-trivial index (not `[1, 2, 3, ... ]`).

In [49]:
#filter by summer 2014
#groupby category

filter_summer_2014 = ((df.month > 5) & (df.month < 9)) & (df.year == 2014)

total_revenue = df[filter_summer_2014].groupby('item_id')['revenue'].sum()

#summer_2014_df['revenue'] = summer_2014_df['item_price']*summer_2014_df['item_cnt_day']
total_revenue.sort_values(ascending=False)

item_id
6675     28485361.70
16790     8018209.20
3928      3768552.05
7807      3740806.20
7172      3356949.20
            ...     
12623           0.00
12640           0.00
7803         -299.00
11367       -1500.00
21775       -3899.00
Name: revenue, Length: 8799, dtype: float64

<ol start="3">
  <li><b>How many items are there, such that their price stays constant (to the best of our knowledge) during the whole period of time?</b></li>
</ol>

* Let's assume, that the items are returned for the same price as they had been sold.

In [52]:
# fix the shop and the item_id, count the prices
df.head()

grouped = df.groupby(['item_id', 'shop_id'])['item_price'].count()

grouped[grouped == 1]

item_id  shop_id
0        54         1
4        54         1
5        54         1
6        54         1
7        54         1
                   ..
22168    2          1
         16         1
         42         1
         43         1
22169    25         1
Name: item_price, Length: 130245, dtype: int64

<ol start="4">
  <li><b>What was the variance of the number of sold items per day sequence for the shop with `shop_id = 25` in December, 2014? Do not count the items, that were sold but returned back later.</b></li>
</ol>

* Fill `total_num_items_sold` and `days` arrays, and plot the sequence with the code below.
* Then compute variance. Remember, there can be differences in how you normalize variance (biased or unbiased estimate, see [link](https://math.stackexchange.com/questions/496627/the-difference-between-unbiased-biased-estimator-variance)). Compute ***unbiased*** estimate (use the right value for `ddof` argument in `pd.var` or `np.var`). 
* If there were no sales at a given day, ***do not*** impute missing value with zero, just ignore that day

In [54]:
df.head()

,date_block_num,shop_id,item_id,item_price,item_cnt_day,day,month,year,revenue
0,0,59,22154,999.00,1.0,2,1,2013,999.00
1,0,25,2552,899.00,1.0,3,1,2013,899.00
2,0,25,2552,899.00,-1.0,5,1,2013,-899.00
3,0,25,2554,1709.05,1.0,6,1,2013,1709.05
4,0,25,2555,1099.00,1.0,15,1,2013,1099.00


In [60]:
test = df[(df.shop_id == 25) & (df.month == 12) & (df.year == 2014)][['item_id', 'item_cnt_day', 'day']].groupby(['day', 'item_id'])['item_id'].count()

test[test != 1]

Series([], Name: item_id, dtype: int64)

In [66]:
df[(df.shop_id == 25) & (df.month == 12) & (df.year == 2014)][['item_id', 'item_cnt_day', 'day']]['item_cnt_day'].value_counts(dropna=False)

 1.0      7121
 2.0      1004
 3.0       284
 4.0       112
 5.0        47
 6.0        32
-1.0        13
 7.0        12
 8.0        10
 9.0         8
 11.0        4
 10.0        4
 20.0        3
 12.0        3
 23.0        3
 18.0        3
 22.0        2
 15.0        2
 29.0        2
 32.0        1
 16.0        1
 63.0        1
 24.0        1
 25.0        1
 33.0        1
 205.0       1
 49.0        1
 17.0        1
 67.0        1
 34.0        1
 72.0        1
 37.0        1
 55.0        1
 14.0        1
Name: item_cnt_day, dtype: int64

In [70]:

int_df = df[(df.shop_id == 25) & (df.month == 12) & (df.year == 2014)][['item_id', 'item_cnt_day', 'day']].groupby('day')[['item_cnt_day']].sum()

In [71]:
int_df

,item_cnt_day
day,
1,153.0
2,169.0
3,195.0
4,164.0
5,300.0
6,350.0
7,325.0
8,209.0
9,204.0


In [72]:
int_df['day'].?

Object `var` not found.


In [76]:
int_df['item_cnt_day'].var()

117167.70229885059

In [73]:
pd.var?

Object `pd.var` not found.


In [74]:
pd.read_csv?